In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose


## Load and Save Data

In [14]:
def load_selected_PM_data(data_path):
    PM_data = {}
    data_name_lst = [name for name in os.listdir(data_path) if name.startswith('Fill')]
    for name in data_name_lst:
        a_df = pd.read_csv(os.path.join(data_path, name), sheet_name='PM2.5')
        PM_data[name.split('_')[1]] = a_df
    return PM_data

def save_to_csv(PM_data, save_path, save_name):
    for key, val in PM_data.items():
        val.to_csv(os.path.join(save_path, f'{save_name}_{key}_.csv'), index=False)

## Compute

In [15]:
def match_label_station(PM_data_year, label_df):
    label_dict = {'Date':PM_data_year['Date']}
    station_columns = PM_data_year.columns[1:]
    for station_id in station_columns:
        the_label = label_df['label'][label_df['station_id'] == station_id].to_list()[0]
        if str(the_label) not in label_dict.keys():
            label_dict[str(the_label)] = PM_data_year[station_id]
        else:
            label_dict[str(the_label)] = pd.concat([label_dict[str(the_label)], PM_data_year[station_id]], axis=1)
    return label_dict

def match_label_select_station(PM_data_year, label_df):
    label_dict = {'Date':PM_data_year['Date']}
    station_columns = PM_data_year.columns[1:]
    for station_id in station_columns:
        the_label = label_df['label'][label_df['station_id'] == station_id].to_list()
        if len(the_label) == 0:
            continue
        else:
            the_label = the_label[0]
        if str(the_label) not in label_dict.keys():
            label_dict[str(the_label)] = PM_data_year[station_id]
        else:
            label_dict[str(the_label)] = pd.concat([label_dict[str(the_label)], PM_data_year[station_id]], axis=1)
    return label_dict

def merge_to_one_year(label_dict):
    new_label_dict =label_dict.copy()
    all_val = 0
    if 'Date' in new_label_dict.keys():
        del new_label_dict['Date']
    for key, df in new_label_dict.items():
        if len(df.shape) > 1:
            mean_data = df.mean(axis=1, numeric_only=True).to_numpy()
        else:
            mean_data = df.to_numpy()
        if mean_data.shape[0] == 365:
            mean_data = np.delete(mean_data, 58)
        all_val += mean_data
    return all_val / len(new_label_dict.keys())

def merge_all_year_as_timeseries(PM_data):
    all_year = pd.DataFrame(columns=['Date', 'Mean PM2.5'])
    for key, df in PM_data.items():
        mean_data = df.mean(axis=1, numeric_only=True)
        a_year_df = pd.DataFrame(df['Date'].copy().to_list(), columns=['Date'])
        a_year_df['Mean PM2.5'] = mean_data.copy().to_list()
        all_year = pd.concat([all_year, a_year_df], axis=0)

    # Add some detail
    all_year = all_year.reset_index().drop(columns=['index'])
    all_year['Date'] = all_year['Date'].apply(lambda x : datetime.strptime(x, '%Y_%m_%d'))
    all_year = all_year.drop_duplicates()
    all_year = all_year.set_index('Date').asfreq(freq='D')
    all_year['Mean PM2.5'] = all_year['Mean PM2.5'].interpolate('linear')
    return all_year

## Label the Data

In [16]:
def meteorology_labeling(provide_label_path):
    NE = ' อำนาจเจริญ, บึงกาฬ, บุรีรัมย์, ชัยภูมิ, กาฬสินธุ์, ขอนแก่น, เลย, มหาสารคาม, มุกดาหาร, นครพนม, นครราชสีมา, หนองบัวลำภู, หนองคาย, ร้อยเอ็ด, สกลนคร, ศรีสะเกษ, สุรินทร์, อุบลราชธานี, อุดรธานี, ยโสธร'
    N = ' เชียงใหม่, เชียงราย, ลำปาง, ลำพูน, แม่ฮ่องสอน, น่าน, พะเยา, แพร่, อุตรดิตถ์, ตาก, สุโขทัย, พิษณุโลก, พิจิตร, กำแพงเพชร, เพชรบูรณ์'
    C = ' นครสวรรค์, อุทัยธานี, อ่างทอง, ชัยนาท, พระนครศรีอยุธยา, กทม., ลพบุรี, นครปฐม, นนทบุรี, ปทุมธานี, สมุทรปราการ, สมุทรสาคร, สมุทรสงคราม, สระบุรี, สิงห์บุรี, สุพรรณบุรี, กาญจนบุรี, ราชบุรี'
    E = ' ฉะเชิงเทรา, จันทบุรี, ชลบุรี, ปราจีนบุรี, ระยอง, สระแก้ว, ตราด, นครนายก'
    SE = ' เพชรบุรี, ประจวบคีรีขันธ์, ประจวบคิรีขันธ์, ชุมพร, นครศรีธรรมราช, นราธิวาส, ปัตตานี, พัทลุง, สงขลา, สุราษฏร์ธานี, ยะลา'
    SW = ' กระบี่, พังงา, ภูเก็ต, ระนอง, สตูล, ตรัง'
    Thai_sector = [NE, N, C, E, SE, SW]
    Thai_dict = {f'{j}': ['จ.' + i.split(' ')[-1] if i.split(' ')[-1] != 'กทม.' else i.split(' ')[-1] for i in Thai_sector[j].split(',')] for j in range(len(Thai_sector))}
    swap_Thai_dict = {provide:sector for sector in Thai_dict.keys() for provide in Thai_dict[sector]}
    provide_df = provide_labeling(provide_label_path)
    label_df = provide_df.copy()
    label_df['label'] = label_df['station_name'].apply(lambda x: swap_Thai_dict[x])
    return label_df

def provide_labeling(provide_label_path):
    try:
        return pd.read_csv(provide_label_path)
    except Exception:
        print('Not Found :', provide_label_path, 'in This Directory')
        return -1
    
def season_label(df):
    new_df = df.copy()
    season_mark = [('02_01', '04_30'), ('05_01', '09_30')] # summer and rainy
    season_lst = []
    for i in new_df.index:
        if i in new_df.loc[season_mark[0][0]: season_mark[0][1]].index:
            season_lst.append('summer')
        elif i in new_df.loc[season_mark[1][0]: season_mark[1][1]].index:
            season_lst.append('rainy')
        else:
            season_lst.append('winter')
    new_df['season'] = season_lst
    return new_df

def avg_season_labeling(PM_data):
    new_PM_data = {}
    for year, df in PM_data.copy().items():
        df['Month_day'] = df['Date'].apply(lambda x: '_'.join(x.split('_')[1:]))
        df = df.set_index(['Month_day'])
        new_df = pd.DataFrame({'Date':df['Date'], 'Average_PM2.5':df.mean(axis=1, numeric_only=True)}, index=df.index)
        new_df = season_label(new_df)
        new_df = new_df.reset_index().drop(columns=['Month_day'])
        new_PM_data[year] = new_df
    return new_PM_data

# Visualization

In [17]:
def plot_PM_mean_curve(label_dict, label_sector=False, save_path=None, plot_title=None):
    label_sector_dict = {'0':'NE', '1':'N', '2':'C', '3':'E', '4':'SE', '5':'SW'}
    new_label_dict =label_dict.copy()
    if 'Date' in new_label_dict.keys():
        del new_label_dict['Date']
    for key, df in new_label_dict.items():
        if len(df.shape) > 1:
            mean_data = df.mean(axis=1, numeric_only=True)
        else:
            mean_data = df
        if label_sector:
            plt.plot(mean_data, label=label_sector_dict[str(key)])
        else:
            plt.plot(mean_data, label=str(key))
    plt.hlines(50, 0, 356, linestyles='dashed', label='Thailand NAAQ (50)', color='gray')
    plt.hlines(15, 0, 356, linestyles='dashed', label='WHO guideline (15)', color='black')
    plt.ylabel('PM2.5 micrograms/m**3')
    plt.xlabel('Day')
    plt.legend()
    if plot_title:
        plt.title(plot_title)
    if save_path:
        plt.savefig(save_path)
    plt.show()

def plot_TS_all_year(TS_PM_data, model='additive', plot_result=False, save_path=None):
    result = seasonal_decompose(TS_PM_data, model='multiplicative')
    if plot_result:
        result.plot()
    trend_data = result.trend.copy().interpolate('linear')
    trend_data = trend_data.fillna(trend_data.to_numpy().mean())
    time_index = trend_data.index.to_numpy()
    trend_df = pd.DataFrame({'Date':time_index, 'PM2.5':trend_data.to_numpy()})
    trend_df['Group'] = trend_df['Date'].apply(lambda x: datetime.strftime(x, '%m_%d'))
    trend_df.groupby(['Group']).mean().plot()
    plt.hlines(50, 0, 356, linestyles='dashed', label='Thailand NAAQ (50)', color='black')
    plt.hlines(15, 0, 356, linestyles='dashed', label='WHO guideline (15)', color='red')
    plt.ylabel('PM2.5 micrograms/m**3')
    plt.xlabel('Day')
    plt.legend()
    if save_path:
        plt.savefig(save_path)
    plt.show()

def plot_TS_3_seasons(TS_PM_data, model='additive', plot_result=False, save_path=None):
    result = seasonal_decompose(TS_PM_data, model='multiplicative')
    if plot_result:
        result.plot()
    trend_data = result.trend.copy().interpolate('linear')
    trend_data = trend_data.fillna(trend_data.to_numpy().mean())
    time_index = trend_data.index.to_numpy()
    trend_df = pd.DataFrame({'Date':time_index, 'PM2.5':trend_data.to_numpy()})
    trend_df['Group'] = trend_df['Date'].apply(lambda x: datetime.strftime(x, '%m_%d'))
    overall_trends = trend_df.groupby(['Group']).mean()

    # Addition for 3 seasons plotting
    hot_season = overall_trends.loc['02_01':'05_01'].copy()
    rain_season = overall_trends.loc['05_01':'10_01'].copy()
    cold_season = pd.concat([overall_trends.loc['10_01':].copy(), overall_trends.loc[:'02_02'].copy()])
    hot_season['IDX'] = np.arange(hot_season.index.shape[0])
    rain_season['IDX'] = np.arange(rain_season.index.shape[0])
    cold_season['IDX'] = np.arange(cold_season.index.shape[0])
    hot_center = ['02_15', '03_15', '04_15']
    rain_center = ['05_15', '06_15', '07_15', '08_15', '09_15']
    cold_center = ['10_15', '11_15', '12_15', '01_15']
    hot_idx = list(map(lambda x: int(hot_season.loc[x]['IDX']), hot_center))
    rain_idx = list(map(lambda x: int(rain_season.loc[x]['IDX']), rain_center))
    cold_idx = list(map(lambda x: int(cold_season.loc[x]['IDX']), cold_center))
    hot_season = hot_season.drop(columns=['IDX'])
    rain_season = rain_season.drop(columns=['IDX'])
    cold_season = cold_season.drop(columns=['IDX'])

    hot_season.plot()
    plt.xticks(hot_idx, labels=hot_center)
    plt.ylim([0, 50])
    plt.grid()
    plt.title('Summer')
    plt.ylabel('PM2.5 micrograms/m**3')
    plt.xlabel('Day')
    plt.legend()
    if save_path:
        plt.savefig(save_path[0])

    rain_season.plot()
    plt.xticks(rain_idx, labels=rain_center)
    plt.ylim([0, 50])
    plt.grid()
    plt.title('Rainy')
    plt.ylabel('PM2.5 micrograms/m**3')
    plt.xlabel('Day')
    plt.legend()
    if save_path:
        plt.savefig(save_path[1])

    cold_season.plot()
    plt.xticks(cold_idx, labels=cold_center)
    plt.ylim([0, 50])
    plt.grid()
    plt.title('Winter')
    plt.ylabel('PM2.5 micrograms/m**3')
    plt.xlabel('Day')
    plt.legend()
    if save_path:
        plt.savefig(save_path[2])

    plt.show()

    

In [18]:
def a_year_pollution_ratio(PM_data_year, cond_lst):
    PM_data_year['Month'] = PM_data_year['Date'].apply(lambda x: x.split('_')[1])
    month_series = PM_data_year['Month'].copy()
    value_df = PM_data_year.copy().drop(columns=['Month', 'Date'])
    count_per_month_df = pd.DataFrame(month_series.unique(), columns=['Month'])
    for idx in range(1, len(cond_lst)):
        upper = cond_lst[idx]
        lower = cond_lst[idx - 1]
        conditional_df = value_df.applymap(lambda x: 1 if x >= lower and x < upper else 0)
        val = pd.concat([month_series, pd.Series(conditional_df.sum(axis=1), name=f'{lower}<=x<{upper}')], axis=1).groupby(['Month']).sum()
        count_per_month_df = pd.concat([count_per_month_df, val[f'{lower}<=x<{upper}'].reset_index()], axis=1)
        count_per_month_df = count_per_month_df.T.drop_duplicates().T
        
    # Normalize
    count_per_month_df = count_per_month_df.astype(np.int32).drop(columns=['Month'])
    sum_count_month = count_per_month_df.copy().sum(axis=1)
    normalized_df = count_per_month_df.div(sum_count_month, axis=0)
    month_df = pd.DataFrame(month_series.unique(), columns=['Month'])
    return pd.concat([month_df, normalized_df.reindex(month_df.index)], axis=1)
    